In [1]:
import torch
import torch.nn as nn

import read_trace_code
import tools_on_B_graph as Btools
from importlib import reload

# Test on mymod :

In [18]:
class SubMod(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Sequential (
            nn.ReLU(),
            nn.Linear(128,512),
        )
    
    def forward(self,p1,p2,p3):
        y1 = self.linear_stack(p1)
        y2 = self.linear_stack(p2)
        y3 = self.linear_stack(p3)
        return y1+y2,y1-y2+y3,y1

class MyMod(nn.Module):
    def __init__(self):
        super().__init__()
        self.op = SubMod()
    def forward(self,x):
        (z1,z2,z3) = self.op(x,-x,p3=torch.ones_like(x))
        return z1 + 2*z3 + z2*x.size(-1)

mymod = MyMod()

In [19]:
src_mymod = torch.rand((128))
script_mymod = torch.jit.trace_module(mymod, {'forward': (src_mymod,)},check_trace=False)
print("===== MYMOD =====")
print(script_mymod.forward.code)

print("===== SUB CODE : self.op.forward =====")
print(script_mymod.op.code)

print("===== SUB SUB CODE =====")
print(script_mymod.op.linear_stack.forward1.code)

print("===== SUB SUB SUB CODE =====")
print("== to debug : getattr(self.op.linear_stack,\"1\").forward2 ==")
print(getattr(script_mymod.op.linear_stack,"1").forward2.code)

===== MYMOD =====
def forward(self,
    x: Tensor) -> Tensor:
  op = self.op
  input = torch.neg(x)
  input0 = torch.ones_like(x, dtype=6, layout=0, device=torch.device("cpu"), pin_memory=False)
  _0, _1, _2, = (op).forward(x, input, input0, )
  _3 = torch.add(_1, torch.mul(_0, CONSTANTS.c0))
  _4 = ops.prim.NumToTensor(torch.size(x, -1))
  return torch.add(_3, torch.mul(_2, _4))

===== SUB CODE : self.op.forward =====
def forward(self,
    x: Tensor,
    input: Tensor,
    input0: Tensor) -> Tuple[Tensor, Tensor, Tensor]:
  linear_stack = self.linear_stack
  _0 = (linear_stack).forward(x, )
  _1 = (linear_stack).forward1(input, )
  _2 = (linear_stack).forward2(input0, )
  z1 = torch.add(_0, _1)
  z2 = torch.add(torch.sub(_0, _1), _2)
  return (_0, z1, z2)

===== SUB SUB CODE =====
def forward1(self,
    input: Tensor) -> Tensor:
  _1 = getattr(self, "1")
  _0 = getattr(self, "0")
  _2 = (_1).forward1((_0).forward1(input, ), )
  return _2

===== SUB SUB SUB CODE =====
== to debug : get

In [20]:
reload(read_trace_code)
reload(Btools)
mymod_Bg = read_trace_code.main(mymod,(src_mymod,))
mymod_Dg = Btools.B_to_D(mymod_Bg)
Btools.print_code(mymod_Dg)

def main(x):
	__10_fv = torch.relu(x)
	__13_y1 = torch.nn.functional.linear(__10_fv,self.op.linear_stack[1].weight,self.op.linear_stack[1].bias)
	__2_input = torch.neg(x)
	__18_fv = torch.relu(__2_input)
	__21_y2 = torch.nn.functional.linear(__18_fv,self.op.linear_stack[1].weight,self.op.linear_stack[1].bias)
	__30_z1 = torch.add(__13_y1,__21_y2)
	__32_fv = torch.sub(__13_y1,__21_y2)
	__4_fv = torch.device("cpu")
	__3_input0 = torch.ones_like(x,device = __4_fv,pin_memory = False)
	__26_fv = torch.relu(__3_input0)
	__29_y3 = torch.nn.functional.linear(__26_fv,self.op.linear_stack[1].weight,self.op.linear_stack[1].bias)
	__31_z2 = torch.add(__32_fv,__29_y3)
	__33_fv = (__13_y1,__30_z1,__31_z2)
	__35__1 = __33_fv[1]
	__34__0 = __33_fv[0]
	__38_fv = torch.mul(__34__0,tensor(2))
	__37__3 = torch.add(__35__1,__38_fv)
	__36__2 = __33_fv[2]
	__39__4 = torch.Tensor.size(x,-1)
	__40_fv = torch.mul(__36__2,__39__4)
	__41_fv = torch.add(__37__3,__40_fv)
	return __41_fv


In [21]:
reload(Btools)
print("== Original module ==")
print(mymod(src_mymod)[:10])
print("== Through D_graph ==")
print(Btools.test_code(mymod_Dg,mymod,{"x":src_mymod})[:10])

== Original module ==
tensor([ 109.5877,  -16.9878, -108.7730,   90.9153, -121.8676,  -25.9335,
         153.3468,  -78.8924,   31.2294,  136.2355], grad_fn=<SliceBackward0>)
== Through D_graph ==
tensor([ 109.5877,  -16.9878, -108.7730,   90.9153, -121.8676,  -25.9335,
         153.3468,  -78.8924,   31.2294,  136.2355], grad_fn=<SliceBackward0>)


# Test on torch.nn.Transformer

In [6]:
mod_tf = nn.Transformer(nhead=16, num_encoder_layers=1)
src = torch.rand((10, 32, 512))
tgt = torch.rand((10, 32, 512))
modtf_Bg = read_trace_code.main(mod_tf,(src,tgt),show_debug=True)
modtf_Dg = Btools.B_to_D(modtf_Bg)
Btools.print_code(modtf_Dg)

In self.forward try to sub open self.encoder.forward
In self.encoder.forward try to sub open self.encoder.layers[0].forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].self_attn.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].dropout1.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].norm1.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].linear1.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].dropout.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].linear2.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].dropout2.forward
In self.encoder.layers[0].forward try to sub open self.encoder.layers[0].norm2.forward
In self.encoder.forward try to sub open self.encoder.norm.forward
In self.forward try to sub open self.decoder.forward
In self.decoder.forward try to sub open self.

# Test on GPT2 made from scratch with random and nlayers=8

In [3]:
import torch
import math
import copy
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import ModuleList
from torch.nn.modules.normalization import LayerNorm

class Conv1D(nn.Module):
    def __init__(self, nx, nf):
        super().__init__()
        self.nf = nf
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(nf))

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(size_out)
        return x
    
# Examples :
"""
d_model = 768
conv1d  = Conv1D(d_model, d_model*3)
x       = torch.rand(1,4,d_model) #represents a sequence of batch_size=1, seq_len=4 and embedding_sz=768, something like "Hello how are you"
x       = conv1d(x)
print(x.shape)

query, key, value = x.split(d_model, dim=-1)

print(query.shape, key.shape, value.shape)
"""

class FeedForward(nn.Module):
    def __init__(self, dropout, d_model=768, nx=768*4):
        super().__init__()
        self.c_fc    = Conv1D(d_model, nx)
        self.c_proj  = Conv1D(nx, d_model)
        self.act     = F.gelu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.dropout(self.c_proj(self.act(self.c_fc(x))))
    
class Attention(nn.Module):
    def __init__(self, d_model=768, n_head=12, n_ctx=1024, d_head=64, bias=True, scale=False):
        super().__init__()
        self.n_head  = n_head
        self.d_model = d_model
        self.c_attn  = Conv1D(d_model, d_model*3)
        self.scale   = scale
        self.softmax = nn.Softmax(dim=-1)
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))
        self.dropout = nn.Dropout(0.1)
        self.c_proj  = Conv1D(d_model, d_model)
        
    def split_heads(self, x):
        "return shape [`batch`, `head`, `sequence`, `features`]"
        new_shape = x.size()[:-1] + (self.n_head, x.size(-1)//self.n_head) 
        x = x.view(new_shape)
        return x.permute(0, 2, 1, 3) 
    
    def _attn(self, q, k, v, attn_mask=None):
        scores  = torch.matmul(q, k.transpose(-2, -1))
        if self.scale: scores = scores/math.sqrt(v.size(-1))
        nd, ns  = scores.size(-2), scores.size(-1)
        if attn_mask is not None: scores = scores + attn_mask
        scores  = self.softmax(scores)
        scores  = self.dropout(scores)
        outputs = torch.matmul(scores, v)
        return outputs
    
    def merge_heads(self, x):
        x         = x.permute(0, 2, 1, 3).contiguous()
        new_shape = x.size()[:-2] + (x.size(-2)*x.size(-1),)
        return x.view(new_shape)
        
    def forward(self, x):
        x        = self.c_attn(x) #new `x` shape - `[1,3,2304]`
        q, k, v  = x.split(self.d_model, dim=2)
        q, k, v  = self.split_heads(q), self.split_heads(k), self.split_heads(v)
        out      = self._attn(q, k, v)
        out      = self.merge_heads(out)
        out      = self.c_proj(out)
        return out
    
class TransformerBlock(nn.Module):
    def __init__(self, d_model=768, n_head=12, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attn        = Attention(d_model=768, n_head=12, d_head=64, n_ctx=1024, bias=True, scale=False)
        self.feedforward = FeedForward(dropout=0.1, d_model=768, nx=768*4)
        self.ln_1        = LayerNorm(d_model)
        self.ln_2        = LayerNorm(d_model)
                
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.feedforward(self.ln_2(x))
        return x
    
def _get_clones(module, n):
    return ModuleList([copy.deepcopy(module) for i in range(n)])

class GPT2(nn.Module):
    def __init__(self, nlayers=12, n_ctx=1024, d_model=768, vcb_sz=50257):
        super(GPT2, self).__init__()
        self.nlayers = nlayers
        block        = TransformerBlock(d_model=768, n_head=12, dropout=0.1)
        self.h       = _get_clones(block, 12)
        self.wte     = nn.Embedding(vcb_sz, d_model)
        self.wpe     = nn.Embedding(n_ctx, d_model)
        self.drop    = nn.Dropout(0.1)
        self.ln_f    = LayerNorm(d_model)
        self.out     = nn.Linear(d_model, vcb_sz, bias=False)
        self.loss_fn = nn.CrossEntropyLoss()
        self.init_weights()
    
    def init_weights(self):
        self.out.weight = self.wte.weight
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, Conv1D)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, Conv1D)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, src, labels=None, pos_ids=None):
        if pos_ids is None: pos_ids = torch.arange(0, src.size(-1)).unsqueeze(0)
        inp = self.drop((self.wte(src)+self.wpe(pos_ids)))
        for i in range(self.nlayers): inp = self.h[i](inp)
        inp     = self.ln_f(inp)
        logits  = self.out(inp)
        outputs = (logits,) + (inp,)
        
        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = self.loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (loss,) + outputs
            return outputs
        return logits
    

In [11]:
model = GPT2(nlayers=8)

In [12]:

model.eval()
# load pretrained_weights from hugging face
# download file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin to `.`

model_dict = model.state_dict() #currently with random initialization
state_dict = torch.load("./gpt2-pytorch_model.bin") #pretrained weights

old_keys = []
new_keys = []
for key in state_dict.keys(): 
    if "mlp" in key: #The hugging face state dict references the feedforward network as mlp, need to replace to `feedforward` be able to reuse these weights
        new_key = key.replace("mlp", "feedforward")
        new_keys.append(new_key)
        old_keys.append(key)

for old_key, new_key in zip(old_keys, new_keys): 
    state_dict[new_key]=state_dict.pop(old_key)

pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict}

model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [13]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
context1  = torch.tensor([tokenizer.encode("The planet earth")])
context2  = torch.tensor([tokenizer.encode("I'm upset with those tools")])
model.forward(context1)

tensor([[[ -5.6025,  -5.6418,  -8.2597,  ..., -11.7372, -11.0343,  -5.3123],
         [-25.5171, -27.3682, -31.0964,  ..., -44.4331, -31.5699, -32.2924],
         [-20.1869, -21.9760, -26.3721,  ..., -43.0404, -27.2938, -26.8470]]],
       grad_fn=<UnsafeViewBackward0>)

In [14]:
jit_tr_GPT2 = torch.jit.trace_module(model, inputs={"forward":context1},check_trace=False)
print(getattr(getattr(jit_tr_GPT2,"h"),"0").code)
reload(read_trace_code)
reload(Btools)
GPT2mod_Bg = read_trace_code.main(model,(context1,),show_debug=True)
GPT2mod_Dg = Btools.B_to_D(GPT2mod_Bg)
Btools.print_code(GPT2mod_Dg)
#print(Btools.test_code(GPT2mod_Dg,model,{"src":context1}))

/tmp/ipykernel_16439/3700145435.py:62: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  new_shape = x.size()[:-1] + (self.n_head, x.size(-1)//self.n_head)


def forward(self,
    argument_1: Tensor) -> Tensor:
  feedforward = self.feedforward
  ln_2 = self.ln_2
  attn = self.attn
  ln_1 = self.ln_1
  _0 = (attn).forward((ln_1).forward(argument_1, ), )
  input = torch.add(argument_1, _0)
  _1 = (feedforward).forward((ln_2).forward(input, ), )
  return torch.add(input, _1)

In self.forward try to sub open self.wte.forward
In self.forward try to sub open self.wpe.forward
In self.forward try to sub open self.drop.forward
In self.forward try to sub open self.h[0].forward
In self.h[0].forward try to sub open self.h[0].ln_1.forward
In self.h[0].forward try to sub open self.h[0].attn.forward
In self.h[0].attn.forward try to sub open self.h[0].attn.c_attn.forward
In self.h[0].attn.forward try to sub open self.h[0].attn.softmax.forward
In self.h[0].attn.forward try to sub open self.h[0].attn.dropout.forward
In self.h[0].attn.forward try to sub open self.h[0].attn.c_proj.forward
In self.h[0].forward try to sub open self.h[0].ln_2.forward
In self.h[0]

# Deterministic GPT2 from scratch n_layer=1, tests OK 

In [22]:
class Conv1D(nn.Module):
    def __init__(self, nx, nf):
        super().__init__()
        self.nf = nf
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(nf))

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(size_out)
        return x
    
    
class FeedForward(nn.Module):
    def __init__(self, dropout, d_model=768, nx=768*4):
        super().__init__()
        self.c_fc    = Conv1D(d_model, nx)
        self.c_proj  = Conv1D(nx, d_model)
        self.act     = F.gelu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.dropout(self.c_proj(self.act(self.c_fc(x))))
    
    
class Attention(nn.Module):
    def __init__(self, d_model=768, n_head=12, n_ctx=1024, d_head=64, bias=True, scale=False, dropout=0.1):
        super().__init__()
        self.n_head  = n_head
        self.d_model = d_model
        self.c_attn  = Conv1D(d_model, d_model*3)
        self.scale   = scale
        self.softmax = nn.Softmax(dim=-1)
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))
        self.dropout = nn.Dropout(dropout)
        self.c_proj  = Conv1D(d_model, d_model)
        
    def split_heads(self, x):
        "return shape [`batch`, `head`, `sequence`, `features`]"
        new_shape = x.size()[:-1] + (self.n_head, x.size(-1)//self.n_head) 
        x = x.view(new_shape)
        return x.permute(0, 2, 1, 3) 
    
    def _attn(self, q, k, v, attn_mask=None):
        scores  = torch.matmul(q, k.transpose(-2, -1))
        if self.scale: scores = scores/math.sqrt(v.size(-1))
        nd, ns  = scores.size(-2), scores.size(-1)
        if attn_mask is not None: scores = scores + attn_mask
        scores  = self.softmax(scores)
        scores  = self.dropout(scores)
        outputs = torch.matmul(scores, v)
        return outputs
    
    def merge_heads(self, x):
        x         = x.permute(0, 2, 1, 3).contiguous()
        new_shape = x.size()[:-2] + (x.size(-2)*x.size(-1),)
        return x.view(new_shape)
        
    def forward(self, x):
        x        = self.c_attn(x) #new `x` shape - `[1,3,2304]`
        q, k, v  = x.split(self.d_model, dim=2)
        q, k, v  = self.split_heads(q), self.split_heads(k), self.split_heads(v)
        # out      = self._attn(q, k, v)
        scores  = torch.matmul(q, k.transpose(-2, -1))
        # if self.scale: scores = scores/math.sqrt(v.size(-1))
        nd, ns  = scores.size(-2), scores.size(-1)
        # if attn_mask is not None: scores = scores + attn_mask
        scores  = self.softmax(scores)
        scores  = self.dropout(scores)
        out = torch.matmul(scores, v)
        
        out      = self.merge_heads(out)
        out      = self.c_proj(out)
        return out
    
class TransformerBlock(nn.Module):
    def __init__(self, d_model=768, n_head=12, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attn        = Attention(d_model=d_model, n_head=n_head, d_head=64, n_ctx=1024, bias=True, scale=False, dropout=dropout)
        self.feedforward = FeedForward(dropout=dropout, d_model=d_model, nx=d_model*4)
        self.ln_1        = LayerNorm(d_model)
        self.ln_2        = LayerNorm(d_model)
                
    def forward(self, x):
        x1 = self.ln_1(x)
        x2 = self.ln_2(x)
        x = x + self.attn(x1)
        x = x + self.feedforward(x2)
        return x
    
def _get_clones(module, n):
    return ModuleList([copy.deepcopy(module) for i in range(n)])

class GPT2(nn.Module):
    def __init__(self, nlayers=12, n_ctx=1024, d_model=768, vcb_sz=50257, dropout=0.1):
        super(GPT2, self).__init__()
        self.nlayers = nlayers
        block        = TransformerBlock(d_model=d_model, n_head=12, dropout=dropout)
        self.h       = _get_clones(block, nlayers)
        self.wte     = nn.Embedding(vcb_sz, d_model)
        self.wpe     = nn.Embedding(n_ctx, d_model)
        self.drop    = nn.Dropout(dropout)
        self.ln_f    = LayerNorm(d_model)
        self.out     = nn.Linear(d_model, vcb_sz, bias=False)
        self.loss_fn = nn.CrossEntropyLoss()
        self.init_weights()
    
    def init_weights(self):
        self.out.weight = self.wte.weight
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, Conv1D)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, Conv1D)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, src, labels=None, pos_ids=None, return_inp=False, dropout=0.1):
        if pos_ids is None: pos_ids = torch.arange(0, src.size(-1)).unsqueeze(0)
        inp = self.drop((self.wte(src)+self.wpe(pos_ids)))
        if return_inp: return inp 
        for i in range(self.nlayers): inp = self.h[0](inp)
        inp     = self.ln_f(inp)
        logits  = self.out(inp)
        outputs = (logits,) + (inp,)
        
        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = self.loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (loss,) + outputs
            return outputs
        return logits

In [23]:
model2 = GPT2(nlayers=1,dropout=0)
model2.eval()
reload(read_trace_code)
reload(Btools)
GPT2mod_Bg2 = read_trace_code.main(model2,(context1,))
GPT2mod_Dg2 = Btools.B_to_D(GPT2mod_Bg2)
Btools.print_code(GPT2mod_Dg2)


def main(src):
	__16_fv = torch.embedding(self.wte.weight,src)
	__10__1 = torch.Tensor.size(src,-1)
	__12_fv = torch.device("cpu")
	__11__2 = torch.arange(0,__10__1,1,dtype = None,device = __12_fv,pin_memory = False)
	__13_input = torch.unsqueeze(__11__2,0)
	__18_fv = torch.embedding(self.wpe.weight,__13_input)
	__14_input0 = torch.add(__16_fv,__18_fv)
	__20_fv = torch.dropout(__14_input0,0.0,False)
	__29_fv = [768]
	__28_x = torch.layer_norm(__20_fv,__29_fv,self.h[0].ln_1.weight,self.h[0].ln_1.bias)
	__47__4 = torch.Tensor.size(__28_x,-1)
	__49_fv = [-1,__47__4]
	__50_fv = torch.Tensor.view(__28_x,__49_fv)
	__48_x = torch.addmm(self.h[0].attn.c_attn.bias,__50_fv,self.h[0].attn.c_attn.weight)
	__43__0 = torch.Tensor.size(__28_x,0)
	__45__2 = torch.Tensor.size(__28_x,1)
	__51_fv = [__43__0,__45__2,2304]
	__52_fv = torch.Tensor.view(__48_x,__51_fv)
	__40__0 = torch.split(__52_fv,768,2)
	__54_x = __40__0[0]
	__57__1 = torch.Tensor.size(__54_x,0)
	__59__3 = torch.Tensor.size(__54_x,1)
	__6

/tmp/ipykernel_16439/4289802552.py:43: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  new_shape = x.size()[:-1] + (self.n_head, x.size(-1)//self.n_head)


In [24]:
reload(Btools)
print("==== Input given to trace ====")
print("== original module ==")
print(model2.forward(context1))
print("== inlined one ==")
print(Btools.test_code(GPT2mod_Dg2,model2,{"src":context1}))
print("==== Different input ====")
print("== original module ==")
print(model2.forward(context2))
print("== inlined one ==")
print(Btools.test_code(GPT2mod_Dg2,model2,{"src":context2}))
print("=== max diff ===")
print(torch.max(torch.abs(model2.forward(context2) - Btools.test_code(GPT2mod_Dg2,model2,{"src":context2}))))

==== Input given to trace ====
== original module ==
tensor([[[ 0.6397, -0.1648,  1.3174,  ..., -0.1638, -0.2060,  0.1671],
         [ 0.8286,  0.2880, -0.0818,  ...,  0.5814, -0.6506,  0.8376],
         [ 0.6993, -0.7192, -0.1985,  ...,  0.3657, -0.7127,  0.6591]]],
       grad_fn=<UnsafeViewBackward0>)
== inlined one ==
tensor([[[ 0.6397, -0.1648,  1.3174,  ..., -0.1638, -0.2060,  0.1671],
         [ 0.8286,  0.2880, -0.0818,  ...,  0.5814, -0.6506,  0.8376],
         [ 0.6993, -0.7192, -0.1985,  ...,  0.3657, -0.7127,  0.6591]]],
       grad_fn=<UnsafeViewBackward0>)
==== Different input ====
== original module ==
tensor([[[ 0.7016, -1.0133,  0.6081,  ..., -0.2734, -0.7781,  0.2357],
         [ 0.5102, -0.9240, -0.1811,  ...,  0.1698, -1.1526,  0.8724],
         [ 0.7101, -1.5677,  0.1003,  ..., -0.1023, -0.7853,  0.1423],
         [ 0.0051, -0.2722,  0.4042,  ..., -0.5499, -0.5823,  0.5711],
         [-0.3649,  0.7561,  1.0420,  ...,  0.0585, -0.5862,  0.6292],
         [ 0.2242,  

# graph test

In [25]:
reload(Btools)
Btools.print_graph(mymod_Dg)


(evince:12087): dbind-WARNING **: 11:47:09.040: Couldn't connect to accessibility bus: Failed to connect to socket /run/user/1000/at-spi/bus: Permission denied


In [ ]:
Btools.print_graph(modtf_Dg)

In [26]:
Btools.print_graph(GPT2mod_Dg)


(evince:12139): dbind-WARNING **: 11:48:42.874: Couldn't connect to accessibility bus: Failed to connect to socket /run/user/1000/at-spi/bus: Permission denied
